<a href="https://colab.research.google.com/github/mahjoubihamza/DeepL/blob/master/Features_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import pickle

In [3]:
def sign(df_var, pct = True):
    df = df_var.copy()
    if pct:
        df = df.pct_change()
    df_sign = pd.DataFrame(columns=df.columns, index=df.index)
    for col in df.columns:
        df_sign[col] = np.where(df[col] > 0, 'pos', 'neg')
    #take NaN values from the original dataframe and put them in the new dataframe
    df_sign[df.isna()] = np.nan
    #take Nan values instead of 0 in the original dataframe and put them in the new dataframe
    df_sign[df == 0] = "no change"
    ##fill nan values with "no change"
    #df_sign = df_sign.fillna('no change')
    return df_sign

In [4]:
!pip install pandas==1.4.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
path_Indices = "/content/drive/MyDrive/Database/Indices.csv"
path_Indicators = "/content/drive/MyDrive/Database/Indicators.csv"
path_Correlations = "/content/drive/MyDrive/Database/Corr_df.csv"


In [6]:
Indices = pd.read_csv(path_Indices)
Indicators = pd.read_csv(path_Indicators)
Corr_df = pd.read_csv(path_Correlations)


In [7]:
Indices.set_index('Date' , inplace = True)

In [8]:
Indices = sign(Indices)

In [9]:
###
Corr_df.drop(columns=['Unnamed: 0'], inplace=True)

In [10]:
Corr_df.groupby('indice').apply(lambda x : x.shape[0])

indice
DAP BULK CFR - INDIA SPOT - USD/T                    1231
DAP BULK FCA - TERNEUZEN/GHENT SPOT - USD/T           553
DAP BULK FOB - US NEW ORLEANS BARGE SPOT - USD/ST     553
MAP GRANULAR BULK CFR - BRAZIL SPOT - USD/T          1230
dtype: int64

In [ ]:
Indicators_ = {}
for indicator in Indicators['Indicator'].unique():
    Indicators_[indicator] = Indicators[Indicators['Indicator'] == indicator]
    Indicators_[indicator].drop(columns=['Indicator', "Unnamed: 0"], inplace=True)
    Indicators_[indicator].set_index('Date', inplace=True)

In [12]:
Indicators = Indicators_
Indicators.keys()

dict_keys(['Spread Fert', '1 week Returns Fert', '2 week Returns Fert', '4 week Returns Fert', '8 week Returns Fert', '13 week Returns Fert', '26 week Returns Fert', 'Vol Fert', '52 week Returns Fert', 'Rank Fert', '1 week Returns IV', '2 week Returns IV', '4 week Returns IV', '8 week Returns IV', '13 week Returns IV', '26 week Returns IV', 'Vol IV', 'Rank IV', '52 week Returns IV', '1 week Returns SA', '2 week Returns SA', '4 week Returns SA', '8 week Returns SA', '13 week Returns SA', '26 week Returns SA', 'Vol SA', 'Rank SA', '52 week Returns SA'])

In [13]:
Corr_df = Corr_df[(Corr_df['nan count'] <= 1) & (Corr_df['2021 - 2023'].isna() == False)]

In [ ]:
Sub_indicators = {}
for indice in Indices.columns:
    df_indice = Corr_df[Corr_df["indice"] == indice]

    Sub_indicators[indice] = pd.DataFrame(index = Indices.index)

    for i in range(len(df_indice)):
        indicator,cat,  feature = df_indice.iloc[i, 1],df_indice.iloc[i, 2], df_indice.iloc[i, 5]
        data_source = " ".join([indicator, cat])
        sub_df = Indicators[data_source][feature]
        Sub_indicators[indice].loc[sub_df.index, "**".join([data_source , feature])] = sub_df